In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from  torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.datasets import KMNIST


In [3]:
train_dataset = KMNIST(root="./KMNIST_date", train=True,  download=True, transform=ToTensor())
test_dataset = KMNIST(root="./KMNIST_date", train=False,  download=True, transform=ToTensor())

100%|██████████| 18.2M/18.2M [00:04<00:00, 4.18MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 754kB/s]
100%|██████████| 3.04M/3.04M [00:00<00:00, 4.62MB/s]
100%|██████████| 5.12k/5.12k [00:00<?, ?B/s]


In [9]:
train_dataset

Dataset KMNIST
    Number of datapoints: 60000
    Root location: ./KMNIST_date
    Split: Train
    StandardTransform
Transform: ToTensor()

In [10]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [12]:
# 定义超参数
lr = 1e-3
epochs = 100
bs = 64

In [13]:
train_dataset_loader = DataLoader(train_dataset,batch_size=bs, shuffle=True)
test_dataset_loader = DataLoader(test_dataset, batch_size=bs, shuffle=True)

In [14]:
# 定义模型
model = nn.Sequential(
    nn.Linear(28*28, 256),
    nn.Sigmoid(),
    nn.Linear(256, 10),
    nn.LogSoftmax(dim=1)
)

# model = nn.Sequential(
#     nn.Linear(28*28, 256),\
#     nn.Sigmoid(),
#     nn.Linear(256, 128),
#     nn.Sigmoid(),
#     nn.Linear(128, 10),
#     nn.LogSoftmax(dim=1)    
# )


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=256, out_features=10, bias=True)
  (3): LogSoftmax(dim=1)
)

In [18]:
torch.cuda.is_available()

True

In [16]:
# 损失函数和优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

In [19]:
# 训练模型
for epoch in range(epochs): 
    for data, target in train_dataset_loader:
        # 用GPU训练
        data = data.view(data.shape[0], -1).to(device)
        target = target.to(device)
        # 前向传播
        output = model(data)
        # 计算损失
        loss = loss_fn(output, target)
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

Epoch 1/100, Loss: 1.4880958795547485
Epoch 2/100, Loss: 1.2073620557785034
Epoch 3/100, Loss: 1.209765076637268
Epoch 4/100, Loss: 1.364089012145996
Epoch 5/100, Loss: 1.314610242843628
Epoch 6/100, Loss: 1.1464968919754028
Epoch 7/100, Loss: 1.1569314002990723
Epoch 8/100, Loss: 1.1701782941818237
Epoch 9/100, Loss: 1.0125271081924438
Epoch 10/100, Loss: 1.0552480220794678
Epoch 11/100, Loss: 1.117464303970337
Epoch 12/100, Loss: 1.0483914613723755
Epoch 13/100, Loss: 1.2671620845794678
Epoch 14/100, Loss: 1.0069644451141357
Epoch 15/100, Loss: 0.8916693925857544
Epoch 16/100, Loss: 0.7249325513839722
Epoch 17/100, Loss: 0.9704926013946533
Epoch 18/100, Loss: 1.0199651718139648
Epoch 19/100, Loss: 0.7837730050086975
Epoch 20/100, Loss: 1.2087364196777344
Epoch 21/100, Loss: 0.9947627782821655
Epoch 22/100, Loss: 0.6963659524917603
Epoch 23/100, Loss: 0.8235871195793152
Epoch 24/100, Loss: 0.7143955230712891
Epoch 25/100, Loss: 0.9724471569061279
Epoch 26/100, Loss: 0.8540990352630615

In [20]:
# 测试模型
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, target in test_dataset_loader:
        data = data.view(data.shape[0], -1).to(device)
        target = target.to(device)
        output = model(data)
        _, predicted = torch.max(output, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f"Accuracy: {100 * correct / total}")

Accuracy: 66.75


In [21]:
predicted

tensor([4, 6, 3, 2, 8, 5, 8, 7, 3, 2, 5, 2, 7, 1, 4, 2], device='cuda:0')

In [22]:
target

tensor([5, 6, 3, 5, 8, 5, 8, 7, 3, 2, 5, 2, 7, 1, 4, 2], device='cuda:0')

In [23]:
predicted == target

tensor([False,  True,  True, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True], device='cuda:0')